### Задание 1, 2
- обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
- при обучении моделей обязательно использовать кроссвалидацию

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv("train_case2.csv", delimiter=';')

In [6]:
df.describe()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,19468.865814,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,2467.251667,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [20]:
# Объединение трансформеров
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score

In [33]:
def classify(classifier, random_state, X_train, y_train, X_test):
    pipeline =  Pipeline([
        ('features',feats),
        ('classifier', classifier(random_state = random_state)),
    ])
    
    #запустим кросс-валидацию
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    
    #обучим пайплайн на всем тренировочном датасете
    pipeline.fit(X_train, y_train)
    y_score = pipeline.predict_proba(X_test)[:, 1]
    
    return cv_score, cv_score_std, y_score

In [40]:
classifiers = [LogisticRegression, GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier]

classify_results = []

for classifier in classifiers:
    classify_results.append({classifier: classify(classifier, 42, X_train, y_train, X_test)})

In [51]:
y_scores = []

for result in classify_results:
    key = next(iter(result))
    cv_score, cv_score_std, y_score = result[key]
    
    y_scores.append(y_score)
    
    print(key)
    print('CV score is {}+-{}'.format(cv_score, cv_score_std))
    print(y_score)
    print('\n')

<class 'sklearn.linear_model._logistic.LogisticRegression'>
CV score is 0.7867401104915408+-0.00852135511666111
[0.21919629 0.24759828 0.40484498 ... 0.16941878 0.70519218 0.75098778]


<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>
CV score is 0.8025125910838183+-0.00707472977074522
[0.1585145  0.19609409 0.36242795 ... 0.13083914 0.73939789 0.78147122]


<class 'sklearn.ensemble._forest.RandomForestClassifier'>
CV score is 0.7734501681056019+-0.007171140345435727
[0.31 0.17 0.03 ... 0.05 0.82 0.73]


<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>
CV score is 0.7952419477681905+-0.0071441451920620375
[0.49276097 0.49343602 0.49719762 ... 0.49260348 0.50477394 0.50811146]




### Задание 3
вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [89]:
from sklearn.metrics import precision_recall_curve, roc_auc_score

def metrics(b, y_test, y_score):
    b=1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(recall)
    return thresholds[ix], fscore[ix], precision[ix], recall[ix]

In [90]:
metrics_result = np.array([metrics(1, y_test, y_score) for y_score in y_scores])

In [91]:
metrics_df = pd.DataFrame(columns=['model', 'precision', 'recall', 'auc', 'f_score'])

metrics_df.model = classifiers
metrics_df.precision = metrics_result[:, 2]
metrics_df.recall = metrics_result[:, 3]
metrics_df.f_score = metrics_result[:, 1]
metrics_df.auc = [roc_auc_score(y_test, y_score) for y_score in y_scores]

metrics_df.sort_values(by='f_score', ascending=False)

,model,precision,recall,auc,f_score
1,<class 'sklearn.ensemble._gb.GradientBoostingC...,0.496852,1.0,0.802615,0.663862
0,<class 'sklearn.linear_model._logistic.Logisti...,0.496170,1.0,0.784035,0.663254
3,<class 'sklearn.ensemble._weight_boosting.AdaB...,0.496028,1.0,0.794572,0.663127
2,<class 'sklearn.ensemble._forest.RandomForestC...,0.496000,1.0,0.771037,0.663102


### Задача 4.
Сделать выводы о том, какая модель справилась с задачей лучше других.

Для задачи предсказания заболевания важнее свести false negative к минимуму. Т.е. recall к максимуму. Градиентный бустинг показал самый высокие f_score и auc.
Если в precision_recall_curve максимизировать f_score, то лучший recall будет у LogReg.

In [95]:
from sklearn.metrics import precision_recall_curve, roc_auc_score

def metrics(b, y_test, y_score):
    b=1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    return thresholds[ix], fscore[ix], precision[ix], recall[ix]

metrics_result = np.array([metrics(1, y_test, y_score) for y_score in y_scores])

metrics_df = pd.DataFrame(columns=['model', 'precision', 'recall', 'auc', 'f_score'])

metrics_df.model = classifiers
metrics_df.precision = metrics_result[:, 2]
metrics_df.recall = metrics_result[:, 3]
metrics_df.f_score = metrics_result[:, 1]
metrics_df.auc = [roc_auc_score(y_test, y_score) for y_score in y_scores]

metrics_df.sort_values(by='recall', ascending=False)

,model,precision,recall,auc,f_score
0,<class 'sklearn.linear_model._logistic.Logisti...,0.647431,0.837558,0.784035,0.730323
2,<class 'sklearn.ensemble._forest.RandomForestC...,0.642669,0.815553,0.771037,0.718863
3,<class 'sklearn.ensemble._weight_boosting.AdaB...,0.692471,0.789401,0.794572,0.737766
1,<class 'sklearn.ensemble._gb.GradientBoostingC...,0.697848,0.788134,0.802615,0.740248


### Задание 5
(опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

PR кривая показывает класс 0, а ROC-AUC - оба класса. Поэтому при дизбалансе лучше работает ROC-AUC.